# WBD Content Intelligence Dashboard

**Goal:** Merge TMDB (metadata), OMDb (IMDb ratings), and Wikipedia Pageviews to surface titles that overperform on interest vs rating.

**Contact:** Pablo Monteros — [GitHub](https://github.com/Pmonteros8) • [LinkedIn](https://www.linkedin.com/in/pmonteros/) • [Email](mailto:Pablo.monterosj@gmail.com)

In [ ]:
import os, requests, pandas as pd, plotly.express as px
from dotenv import load_dotenv
load_dotenv("config/.env")
TMDB_KEY = os.getenv("TMDB_API_KEY")
OMDB_KEY = os.getenv("OMDB_API_KEY")
assert TMDB_KEY, "Missing TMDB_API_KEY in config/.env"
assert OMDB_KEY, "Missing OMDB_API_KEY in config/.env"

def tmdb_search_company(name: str):
    return requests.get(f"https://api.themoviedb.org/3/search/company?api_key={TMDB_KEY}&query={name}").json()

def tmdb_discover_titles(company_id: int, content_type="movie", page=1):
    return requests.get(f"https://api.themoviedb.org/3/discover/{content_type}?api_key={TMDB_KEY}&with_companies={company_id}&page={page}&sort_by=popularity.desc").json()

def tmdb_external_ids(content_type: str, tmdb_id: int):
    return requests.get(f"https://api.themoviedb.org/3/{content_type}/{tmdb_id}/external_ids?api_key={TMDB_KEY}").json()

def omdb_by_imdb_id(imdb_id: str):
    return requests.get(f"http://www.omdbapi.com/?i={imdb_id}&apikey={OMDB_KEY}").json()

def wiki_pageviews(title: str, start="20240101", end="20241231"):
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title.replace(' ','_')}/daily/{start}/{end}"
    r = requests.get(url)
    return [{"date": x["timestamp"][:8], "views": x["views"]} for x in r.json().get("items", [])] if r.status_code==200 else []

## Fetch titles

In [ ]:
company = "Warner Bros"
companies = tmdb_search_company(company).get("results", [])
cid = companies[0]["id"] if companies else None
cid

## Enrich and visualize

In [ ]:
titles = []
for content_type in ["movie","tv"]:
    for page in range(1,4):
        data = tmdb_discover_titles(cid, content_type, page)
        for it in data.get("results", []):
            titles.append({"content_type": content_type, "tmdb_id": it["id"], "title": it.get("title") or it.get("name"), "popularity": it.get("popularity", 0)})
rows = []
for t in titles[:80]:
    ext = tmdb_external_ids(t["content_type"], t["tmdb_id"])
    imdb_id = ext.get("imdb_id")
    rating = None
    if imdb_id:
        om = omdb_by_imdb_id(imdb_id)
        rating = float(om["imdbRating"]) if om.get("imdbRating") not in [None, "N/A"] else None
    wv = wiki_pageviews(t["title"])
    views_total = sum([x["views"] for x in wv]) if wv else None
    rows.append({**t, "imdb_id": imdb_id, "imdb_rating": rating, "wiki_views_2024": views_total})
df = pd.DataFrame(rows)
df.head()